**라이브러리 로드**

In [ ]:
import torch
print("torch.cuda.is_available() =", torch.cuda.is_available())
print("torch.cuda.device_count() =", torch.cuda.device_count())
print("torch.version.cuda =", torch.version.cuda)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

torch.cuda.is_available() = True
torch.cuda.device_count() = 1
torch.version.cuda = 12.6


***라이브러리 설치 - RoboFlow***

In [ ]:
!pip install roboflow ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 75.9 MB/s eta 0:00:00


**YOLO Fine-tuning**

In [ ]:
# ==========================================
# Roboflow 데이터 다운로드
# ==========================================

from roboflow import Roboflow
rf = Roboflow(api_key="     ")
project = rf.workspace("yunhoo93").project("09-yezue-4s43q")
dataset = project.version(5).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to 09-5 in yolov8:: 100%|██████████| 45833/45833 [00:26<00:00, 1712.92it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# PHASE 1 — 일반화 학습

from ultralytics import YOLO

model = YOLO("yolo11x.pt")

model.train(
    data=f"{dataset.location}/data.yaml",

    epochs=180,
    patience=25,

    imgsz=1024,
    batch=8,
    multi_scale=True,

    optimizer="AdamW",
    lr0=0.00038,
    lrf=0.01,
    weight_decay=0.00055,
    warmup_epochs=3,
    cos_lr=True,

    # ✅ 과적합 억제 + 일반화
    mosaic=0.15,       # ← 안정적
    mixup=0.0,         # ← detection에 불리 → 제거
    translate=0.10,
    scale=0.40,
    shear=1.7,
    erasing=0.20,
    close_mosaic=10,
    auto_augment="none",
    copy_paste=0.15,   # ← 소수 클래스 강화

    workers=8,
    project="train_results",
    name="phase1_base",
    resume=False,
    plots=True
)

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=none, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.15, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/09-5/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=180, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00038, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=train_results/phase1_base/weights/last.pt, momentum=0.937, mosaic=0.15, multi_scale=True, name=phase1_base, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patien

In [ ]:
# # PHASE 2 — 고해상도 정밀 박스 회귀 (선택)

# model = YOLO("train_results/phase1_base/weights/best.pt")

# model.train(
#     data=f"{dataset.location}/data.yaml",

#     epochs=70,
#     patience=15,

#     imgsz=1280,
#     rect=True,
#     batch=6,

#     optimizer="AdamW",
#     lr0=0.00014,
#     lrf=0.01,
#     warmup_epochs=2,
#     weight_decay=0.0004,
#     cos_lr=True,
#     box=8.0,
#     dfl=1.7,
#     mosaic=0.0,
#     mixup=0.0,
#     translate=0.05,
#     scale=0.20,
#     erasing=0.12,
#     shear=0.0,
#     perspective=0.0,

#     close_mosaic=1,

#     workers=8,
#     project="train_results",
#     name="phase2_hires",
#     resume=False,
#     plots=True
# )

In [ ]:
# # PHASE 3 — 초고해상도 초저LR 수렴 (선택)

# model = YOLO("train_results/phase2_hires/weights/best.pt")

# model.train(
#     data=f"{dataset.location}/data.yaml",

#     epochs=40,
#     patience=8,

#     imgsz=1792,
#     rect=True,
#     batch=4,

#     optimizer="AdamW",
#     lr0=0.000075,
#     lrf=0.01,
#     warmup_epochs=1,
#     weight_decay=0.0005,
#     cos_lr=True,

#     mosaic=0.0,
#     mixup=0.0,
#     translate=0.05,
#     scale=0.15,
#     erasing=0.10,

#     workers=8,
#     project="train_results",
#     name="phase3_ultra",
#     resume=False,
#     plots=True
# )

In [ ]:
# 런타임이 끊길 경우를 대비한 'pt' 저장 코드 (경로 설정)
from google.colab import files

path = '/content/train_results/phase1_bases/weights/'
files.download(path + 'best.pt')
files.download(path + 'last.pt')

In [ ]:
# ✅ Final Validation
metrics = model.val(
    data=f"{dataset.location}/data.yaml",
    imgsz=1536,          # 속도/성능 균형, 1536이 가장 일반적으로 안정적
    batch=8,
    split="val",         # 검증 데이터셋 명시
    tta=True,            # Test-Time Augmentation (성능 상승 효과 있음)
    plots=True,          # PR curve, confusion matrix 등 자동 저장
    save_json=False      # COCO json 저장 필요 없으면 False
)

# ✅ Overall metrics
print("=== Final Metrics ===")
print(f"mAP50      : {metrics.box.map50:.4f}")
print(f"mAP50-95   : {metrics.box.map:.4f}")
print(f"Precision  : {metrics.box.mp:.4f}")
print(f"Recall     : {metrics.box.mr:.4f}")

# ✅ 클래스별 상세 분석
print("\n=== Per-Class Metrics ===")
for cls, (ap50, ap) in metrics.ap_per_class().items():
    print(f"Class {cls} | AP50: {ap50:.4f}, AP50-95: {ap:.4f}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# ✅ CSV 불러오기
path_result = '/train_results/phase1_bases/results.csv'
df = pd.read_csv(path_result)

# ✅ Epoch 정보 가져오기 (x축)
epochs = df.index  # YOLO results.csv는 epoch = row index

# ✅ 1. LOSS PLOT
plt.figure(figsize=(10, 6))
plt.plot(epochs, df['train/box_loss'], label='box_loss')
plt.plot(epochs, df['train/cls_loss'], label='cls_loss')
plt.plot(epochs, df['train/dfl_loss'], label='dfl_loss')

plt.title("Training Losses")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.show()


# ✅ 2. METRIC PLOT (P, R, mAP50, mAP50-95)
plt.figure(figsize=(10, 6))
plt.plot(epochs, df['metrics/precision(B)'], label='Precision')
plt.plot(epochs, df['metrics/recall(B)'], label='Recall')
plt.plot(epochs, df['metrics/mAP50(B)'], label='mAP50')
plt.plot(epochs, df['metrics/mAP50-95(B)'], label='mAP50-95')

plt.title("Validation Metrics")
plt.xlabel("Epoch")
plt.ylabel("Metric Value")
plt.grid(True)
plt.legend()
plt.show()